In [ ]:
"""
good paper about losses:
https://arxiv.org/pdf/2006.14822.pdf

try freezing the encoder?? what does it mean? --> https://segmentation-models.readthedocs.io/en/latest/tutorial.html
transfer learning in encoder with tumor relared neural network
how to implement dropout
number of parameters in network
resnet34 or densenet121

TODOs:

check the activation of layers --> https://awjuliani.medium.com/visualizing-neural-network-layer-activation-tensorflow-tutorial-d45f8bf7bbc4
implement shuffling in the generator --> https://github.com/qubvel/segmentation_models/blob/master/examples/multiclass%20segmentation%20(camvid).ipynb // https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly 
"""

In [ ]:
# from tensorflow.keras import layers
# from tensorflow import keras
# import segmentation_models as sm

# def get_model_with_dropout(base_model_name='efficientnetb4',activation='sigmoid',dropout = 0.1):
#     base_model = sm.Unet(base_model_name, encoder_weights='imagenet')
#     base_model_input = base_model.input
#     base_model_output = base_model.get_layer('final_conv').output
#     #add dropout
#     base_model_output = keras.layers.Dropout(dropout)(base_model_output)
#     #add activation
#     output = keras.layers.Activation(activation, name=activation)(base_model_output)
#     model_dp = keras.models.Model(base_model_input, output)

#     return model_dp

In [2]:
import os
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

#Run from server

gpu4 = "GPU-71b4cdfc-e381-0b98-9b24-4fc06284b496" 
gpu5 = "GPU-99d0769a-9f86-4800-a40e-2320dddcf5d1" 
gpu6 = "GPU-7423cfb5-cff4-ec4d-7e96-ea6e1591d56f"
gpu7 = "GPU-c0a8738f-6dd0-1b78-c38f-4969fd3886a8"

os.environ["CUDA_VISIBLE_DEVICES"]= gpu6
print(get_available_devices())

RuntimeError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import h5py
import cv2
import time
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, Lambda, GlobalAveragePooling2D, concatenate
from tensorflow.keras.layers import UpSampling2D, Conv2D, Dropout, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

In [ ]:
IMG_HEIGHT = 240
IMG_WIDTH = 240
IMG_DEPTH = 155

IMG_HEIGHT_UNET = 256
IMG_WIDTH_UNET = 256

N_IMG = 369
length_file = IMG_DEPTH * N_IMG
h5py_file_name = 'delete.hdf5'
# h5py_file_name = 'brats2020_full_normalized.hdf5'
smooth = 1e-4

___

Generating the data and creating data arrays

In [ ]:
def create_dataset(path_directory, h5py_file_directory):
    
    def path_extraction(path_directory):
    
        ImgDir = path_directory
        features_t1_path = []
        features_t1ce_path = []
        features_t2_path = []
        features_flair_path = []
        labels_path = []
        count = 0

        for folder in os.listdir(ImgDir):
            count +=1
            if 'Training' in folder:
                new_dir = os.path.join(ImgDir,folder)
                data = os.listdir(new_dir)
                for files in data:
                    if 't1' in files:
                        features_t1_path.append(os.path.join(new_dir, files))
                    if 't1ce' in files:
                        features_t1ce_path.append(os.path.join(new_dir, files))
                    if 't2' in files:
                        features_t2_path.append(os.path.join(new_dir, files))
                    if 'flair' in files:
                        features_flair_path.append(os.path.join(new_dir, files))
                    if 'seg' in files:
                        labels_path.append(os.path.join(new_dir, files))

        #print(len(features_path))
        #print(len(labels_path))
        
        return features_t1_path, features_t1ce_path, features_t2_path, features_flair_path, labels_path
    
    def concatenate_images(path, features = None):
        
        IMG_HEIGHT = 240
        IMG_WIDTH = 240
        IMG_DEPTH = 155
        
        img_conc = np.zeros((len(path),IMG_HEIGHT,IMG_WIDTH,IMG_DEPTH))

        for file, i in zip(path,range(len(path))):
            img = nib.load(file)
            imgarr = img.get_fdata()
            
            if features:
                
#                 mean = np.mean(imgarr)
#                 std = np.std(imgarr)
#                 imgarr_norm = (imgarr - mean) / std
#                 imgarr_norm_clip = np.clip(imgarr_norm,-5,5)
#                 imgarr_std = imgarr_norm_clip/np.max(imgarr_norm_clip)
#                 imgarr = imgarr_std
                
                scaler = MinMaxScaler()
                imgarr_fit = scaler.fit_transform(imgarr)
            
            img_conc[i,:,:,:] = imgarr_fit
            plt.imshow(imgarr_fit[...,75])
            print("min: ", np.min(imgarr_fit, " max: ", np.nax(imgarr_fit)))

        img_conc = np.concatenate(img_conc,axis=2)
        print(np.shape(img_conc_features))
        
        return img_conc
    
    def create_h5py(img_conc_features_t1, img_conc_features_t1ce, img_conc_features_t2, img_conc_features_flair, img_conc_labels, h5py_file_directory):
        
        with h5py.File(h5py_file_directory, 'a') as f:
            f.create_dataset("t1", data=img_conc_features_t1, compression="gzip")
            f.create_dataset("t1ce", data=img_conc_features_t1ce, compression="gzip")
            f.create_dataset("t2", data=img_conc_features_t2, compression="gzip")
            f.create_dataset("flair", data=img_conc_features_flair, compression="gzip")
            f.create_dataset("labels", data=img_conc_labels, compression="gzip")
            
    features_t1_path, features_t1ce_path, features_t2_path, features_flair_path, labels_path = path_extraction(path_directory)
    
    img_conc_features_t1 = concatenate_images(features_t1_path, features = True)
    img_conc_features_t1ce = concatenate_images(features_t1ce_path, features = True)
    img_conc_features_t2 = concatenate_images(features_t2_path, features = True)
    img_conc_features_flair = concatenate_images(features_flair_path, features = True)
    img_conc_labels = concatenate_images(labels_path, features = False)
    
    create_h5py(img_conc_features_t1, img_conc_features_t1ce, img_conc_features_t2, img_conc_features_flair, img_conc_labels, h5py_file_directory)
    

In [ ]:
# path_directory = os.path.join("..","data","MICCAI_BraTS2020_TrainingData")
# dataset_directory = os.path.join("..","data","can_be_deleted.hdf5")

path_directory = os.path.join("..","data","MICCAI_BraTS2020_TrainingData")
dataset_directory = os.path.join("..","data", h5py_file_name)

In [ ]:
#create_dataset(path_directory,dataset_directory)

In [ ]:
hdf5_store_Brats = h5py.File(os.path.join('..','data',h5py_file_name), "r")
test_img = hdf5_store_Brats["features"][:, :, 100]

print(np.min(test_img))
    
plt.imshow(test_img, cmap ='gray')
hdf5_store_Brats.close()

In [ ]:
hdf5_store_Brats = h5py.File(os.path.join('..','data',h5py_file_name), "r")
test_img = hdf5_store_Brats["labels"][:, :, 100]
plt.imshow(test_img, cmap ='gray')
hdf5_store_Brats.close()

___

Load data and model

____

In [ ]:
import segmentation_models as sm

BACKBONE = 'resnet34' #'densenet121' 
CLASSES = ['0','1','2','3','4']
LR = 0.0001
EPOCHS = 70
n_classes = 5
BATCH_SIZE = 32
#decoder_filters = (128, 64, 32, 16, 8)   #standard:(256, 128, 64, 32, 16)

In [ ]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x

def visualize_histories(**images):
    """Import as tuples: one = (a,b)"""
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.ylim(0,1)
        plt.xlabel('epoch')
        if i == 0:
            plt.ylabel('dice_score')
        plt.title(' '.join(name.split('_')).title())
        plt.plot(np.asarray(image[0]))
        plt.plot(np.asarray(image[1]))
        plt.legend(['train', 'val'])
    plt.show()

In [ ]:
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly.html

class SingleDataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=2, dim=(240,240), dim_unet=(256,256), n_channels=3,
                 n_classes=5, classes = CLASSES, shuffle=True):
        'Initialization'
        self.dim = dim
        self.dim_unet = dim_unet
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.classes = classes
        self.class_values = [self.classes.index(cls.lower()) for cls in self.classes]

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        
        # Generate data
        X, y1 = self.__data_generation(list_IDs_temp)

        return X, y1

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        # X = np.zeros((self.batch_size, *self.dim_unet, self.n_channels))
        # y1 = np.zeros((self.batch_size, *self.dim_unet, self.n_classes))
        
#         for i, ID in enumerate(list_IDs_temp):
            

        with h5py.File(os.path.join('..','data',h5py_file_name), "r") as f:
            #images_train = f["features"][()] #the whole dataset: 57195 images
            #labels_train = f["labels"][()]

            images = f["features"]
            labels = f["labels"]

            X_temp = np.zeros((self.batch_size, *self.dim_unet, self.n_channels))
            y_temp = np.zeros((self.batch_size, *self.dim_unet))

            X_temp[:,:IMG_HEIGHT,:IMG_WIDTH,0] = tf.transpose(tf.constant(images[:,:,list_IDs_temp],dtype=tf.float32),[2,0,1])
            X_temp[:,:IMG_HEIGHT,:IMG_WIDTH,1] = X_temp[:,:IMG_HEIGHT,:IMG_WIDTH,0]
            X_temp[:,:IMG_HEIGHT,:IMG_WIDTH,2] = X_temp[:,:IMG_HEIGHT,:IMG_WIDTH,0]

            X_temp = tf.transpose(X_temp, [2, 0, 1, 3])

            
            y_temp[:,:IMG_HEIGHT,:IMG_WIDTH] = tf.transpose(tf.constant(labels[:,:,list_IDs_temp],dtype=tf.float32),[2, 0, 1])
            y_temp = tf.transpose(y_temp, [2, 0, 1])

            # extract certain classes from mask (e.g. cars)
            masks = [(y_temp == v) for v in self.class_values]
            y_temp = np.stack(masks, axis=-1).astype('float')
        
        return X_temp, y_temp

In [ ]:
# define network parameters
n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES))  # case for binary and multiclass segmentation
# n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)
#encoder weights from ImageNet, not pretty realted to tumors, so i delete it, interesting to add other weights

In [ ]:
#calculates dice considering an input with a single class
def dice_single(true,pred):
    true = K.batch_flatten(true)
    pred = K.batch_flatten(pred)
    pred = K.round(pred)

    intersection = K.sum(true * pred, axis=-1)
    true = K.sum(true, axis=-1)
    pred = K.sum(pred, axis=-1)

    return ((2*intersection) + K.epsilon()) / (true + pred + K.epsilon())

def dice_inner_0(true,pred,index=0):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

def dice_inner_1(true,pred,index=1):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

def dice_inner_2(true,pred,index=2):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

def dice_inner_3(true,pred,index=3):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

def dice_inner_4(true,pred,index=4):

    #get only the desired class
    true = true[:,:,:,index]
    pred = pred[:,:,:,index]

    #return dice per class
    return dice_single(true,pred)

metrics_test5 = [dice_inner_0, dice_inner_1, dice_inner_2, dice_inner_3, dice_inner_4, sm.metrics.FScore(threshold=0.5)]

In [ ]:
# define optomizer
optim = tf.keras.optimizers.Adam(LR)
total_loss = sm.losses.DiceLoss(class_weights=np.array([0.1, 1, 1, 0.1, 1])) 

In [ ]:
# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics_test5)

In [ ]:
train_stop = 9001
valid_stop = 10000

params = {'dim_unet': (256,256),
          'batch_size': 32,
          'n_classes': 5,
          'n_channels': 3,
          'shuffle': False}

train_generator = SingleDataGenerator(range(train_stop),**params)
valid_generator = SingleDataGenerator(np.arange(train_stop,valid_stop),**params)

In [ ]:
class printbatch(tf.keras.callbacks.Callback):
    
    def on_train_begin(self, epoch, logs={}):

        res_dir = os.path.join("..","data","test_images_1")

        try:
            os.makedirs(res_dir)
        except:
            print(f"{res_dir} directory already exist")

    
    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        
#         res_dir = os.path.join("..","data","test_images_segmentation_models")
    
#         x_img = os.path.join(res_dir,"X_input.jpg")
#         y_img = os.path.join(res_dir,"Y_truth.jpg")
#         predicted_img = os.path.join(res_dir,f"{epoch}_Y_predicted.jpg")
        
#         image, gt_mask = train_dataset[100]
#         #image = np.expand_dims(image, axis=0)
#         pr_mask = self.model.predict(image[np.newaxis, ...])
    
#         gt_mask_vis = gt_mask[...,0:3]
#         pr_mask_vis = pr_mask[...,0:3]

#         cv2.imwrite(x_img, image[:,:,0])
#         cv2.imwrite(y_img, gt_mask_vis.squeeze())
#         cv2.imwrite(predicted_img, pr_mask_vis.squeeze())
        
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))

pb = printbatch()        

In [ ]:
checkpoint_path =  os.path.join("..","data","checkpoints","training_segmentation_models/cp-{epoch:04d}.ckpt")
checkpoint_dir = os.path.dirname(checkpoint_path)

STEPS_PER_EPOCH = train_stop / BATCH_SIZE
SAVE_PERIOD = 5

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    save_weights_only=True,
    save_freq=int(SAVE_PERIOD * STEPS_PER_EPOCH)
    )

history_callback = tf.keras.callbacks.CSVLogger('history_segmentation_models.csv',separator=',',append=False)

callbacks = [pb, cp_callback, history_callback]


In [ ]:
history = model.fit_generator(
    train_generator, 
    epochs=EPOCHS,
    verbose = 1,
    callbacks = callbacks,
    validation_data=valid_generator
)

___

In [ ]:
visualize_histories(
    class_zero = (history.history['dice_inner_0'], history.history['val_dice_inner_0']), ### if putting the whole image, black and white, could it be the reason of the problem??
    class_one = (history.history['dice_inner_1'], history.history['val_dice_inner_1']),
    class_two = (history.history['dice_inner_2'], history.history['val_dice_inner_2']),
    class_three = (history.history['dice_inner_3'], history.history['val_dice_inner_3']),
    class_four = (history.history['dice_inner_4'], history.history['val_dice_inner_4'])
)

In [ ]:
checkpoint_path= os.path.join("..","data","checkpoints","unet_brats.ckpt")

In [ ]:
model.save_weights(checkpoint_path)
# Restore the weights
# model.load_weights(checkpoint_path)

In [ ]:
# Plot training & validation iou_score values
plt.figure(figsize=(30, 5))
plt.subplot(121)
plt.plot(history.history['f1-score'])
plt.plot(history.history['val_f1-score'])
plt.title('Model f1-score')
plt.ylabel('f1-score')
plt.xlabel('Epoch')
plt.ylim([0,1])
plt.legend(['Train', 'Val'], loc='upper left')

# Plot training & validation loss values
plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.ylim([0,1])
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
n = 1
# ids = np.random.choice(np.arange(len(labels_train)), size=n)
ids = [50]
for i in ids:
    
    image, gt_mask = valid_dataset[i]
    #image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image[np.newaxis, ...])
    
    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[...,1:4].squeeze(),
        pr_mask=pr_mask[...,0:3].squeeze(),
    )


In [ ]:
# model = sm.Unet('resnet34', classes = 5, input_shape=(IMG_HEIGHT_UNET, IMG_WIDTH_UNET, 3))
# model.compile(
#     'Adam',
#     loss=sm.losses.bce_jaccard_loss,
#     metrics=[sm.metrics.iou_score],
# )
# history = model.fit_generator(train_dataloader, epochs = 20, verbose=1, validation_data=None, class_weight=None)

Image visualization of training

In [ ]:
test_img_feat_dir = os.path.join("..","data","MICCAI_BraTS2020_TrainingData","BraTS20_Training_001","BraTS20_Training_001_flair.nii.gz")
img_feat = nib.load(test_img_feat_dir)
imgarr_feat = img_feat.get_fdata()
test_img_feat_slice = imgarr_feat[:,:,100]
test_img = np.zeros((240, 240 ,3), np.float32)

test_img[:,:,0] = test_img_feat_slice
test_img[:,:,1] = test_img_feat_slice
test_img[:,:,2] = test_img_feat_slice

plt.imshow(test_img[:,:,0], cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
directory_predicted_3 = os.path.join("..","data","test_images_1","3_Y_predicted.jpg")
directory_predicted_29 = os.path.join("..","data","test_images","29_Y_predicted.jpg")
directory_predicted_39 = os.path.join("..","data","test_images","39_Y_predicted.jpg")
directory_predicted_49 = os.path.join("..","data","test_images","49_Y_predicted.jpg")
directory_y = os.path.join("..","data","test_images","Y_truth.jpg")
directory_X = os.path.join("..","data","test_images","X_input.jpg")

visualize(
    input_image=cv2.imread(directory_X,0).squeeze(),
    ground_truth=cv2.imread(directory_y,0).squeeze(),
    predicted_3=cv2.imread(directory_predicted_3,0),
    predicted_29=cv2.imread(directory_predicted_29,0),
    predicted_39=cv2.imread(directory_predicted_39,0),
    predicted_49=cv2.imread(directory_predicted_39,0),### if putting the whole image, black and white, could it be the reason of the problem??
    
)


In [ ]:
plot_metrics(history)

____